## Introduction
#### 데이터 분석의 대부분은 데이터를 불러오고, 정제하고, 변형하는 준비에 대부분의 시간이 소모된다. 
#### 이장에서는 결측치, 중복데이터, 문자열 데이터, 그리고 다른 분석적 데이터 변환에 대한 도구들을 다룬다. 

## 누락된 데이터 처리하기 
#### pandas객체의 모든 기술 통계는 누락된 데이터를 배제하고 처리한다.
#### pandas에서는 누락된 데이터를 실수값인 NaN으로 취급한다. 이는 누락된 값을 쉽게 찾을 수 있도록 하는 역할을 한다.

In [1]:
import pandas as pd
import numpy as np

string_data = pd.Series(['aardvak', 'artichoke', np.nan, 'avocado'])
string_data

0      aardvak
1    artichoke
2          NaN
3      avocado
dtype: object

In [2]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

#### NaN은 데이터가 존재하지 않거나, 존재하더라도 데이터를 수집하는 과정에서 검출되지 않았음을 의미한다. 
#### 데이터를 정제하는 과정에서 결측치 자체를 데이터 수집과정에서의 실수나 결측치로 인한 잠재적 편향을 찾아내는 수단으로 인식해야 한다.
#### 표 7-1 NA 처리 메소드 참조

In [3]:
string_data[0] = None

string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

### 누락된 데이터 골라내기 

In [4]:
from numpy import nan as NA

data = pd.Series([1, NA, 3.5, NA, 7]) #Series에서 dropna 메소드를 적용하면 Null이 아닌 데이터와 색인만 Series에 반환한다.
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

In [5]:
data[data.notnull()] #dropna와 동일한 결과

0    1.0
2    3.5
4    7.0
dtype: float64

In [6]:
#DataFRame 객체의 경우 좀 복잡하다. 
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],
                   [NA, NA, NA], [NA, 6.5, 3.]])

cleaned = data.dropna()
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [7]:
cleaned #DataFrame 객체에서 dropna는 기본적으로 NA 값을 하나라도 포함하고 있는 row를 제거한다.

,0,1,2
0,1.0,6.5,3.0


In [8]:
data.dropna(how='all') #how='all' 옵션의 경우 모두 NA인 row만 제거한다.

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


In [9]:
data[4] = NA  #4번째 컬럼에 NaN추가
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [10]:
data.dropna(axis=1, how='all') #컬럼을 제거하는 것도 동일하다. 옵션으로 axis=1을 넘겨준다.

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


#### DataFrame의 row를 제거하는 방법은 시계열에서 주로 사용된다. 몇개 이상의 값이 들어있는 row를 보고 싶으면 thresh 인자를 사용 

In [11]:
df = pd.DataFrame(np.random.randn(7,3))
df.iloc[:4, 1] = NA
df.iloc[:2, 2] = NA

df

,0,1,2
0,-0.007904,NaN,NaN
1,-0.419093,NaN,NaN
2,1.868058,NaN,0.603798
3,-1.282961,NaN,0.621834
4,1.076719,0.452850,-0.995992
5,0.655906,0.804638,-1.080234
6,0.409881,0.493251,-1.179825


In [12]:
df.dropna()

,0,1,2
4,1.076719,0.452850,-0.995992
5,0.655906,0.804638,-1.080234
6,0.409881,0.493251,-1.179825


In [30]:
df.dropna(thresh=6) #결측치를 제외하고 최소 2개 이상인 데이터를 가진것만 것만  

,0,1,2


In [33]:
df.dropna(axis=1, thresh=2)  

,0,1,2
0,-0.007904,NaN,NaN
1,-0.419093,NaN,NaN
2,1.868058,NaN,0.603798
3,-1.282961,NaN,0.621834
4,1.076719,0.452850,-0.995992
5,0.655906,0.804638,-1.080234
6,0.409881,0.493251,-1.179825


### 결측치 채우기 
#### 누락한 값을 제외시키지 않고 데이터상의 '구멍'을 메우고 싶을때, fillna메소드를 사용한다. 

In [34]:
df

,0,1,2
0,-0.007904,NaN,NaN
1,-0.419093,NaN,NaN
2,1.868058,NaN,0.603798
3,-1.282961,NaN,0.621834
4,1.076719,0.452850,-0.995992
5,0.655906,0.804638,-1.080234
6,0.409881,0.493251,-1.179825


In [35]:
df.fillna(0)

,0,1,2
0,-0.007904,0.000000,0.000000
1,-0.419093,0.000000,0.000000
2,1.868058,0.000000,0.603798
3,-1.282961,0.000000,0.621834
4,1.076719,0.452850,-0.995992
5,0.655906,0.804638,-1.080234
6,0.409881,0.493251,-1.179825


In [36]:
df

,0,1,2
0,-0.007904,NaN,NaN
1,-0.419093,NaN,NaN
2,1.868058,NaN,0.603798
3,-1.282961,NaN,0.621834
4,1.076719,0.452850,-0.995992
5,0.655906,0.804638,-1.080234
6,0.409881,0.493251,-1.179825


In [37]:
df.fillna({1:0.5, 2: 0}) #fillna에 사전값을 넘겨서 컬럼마다 다른값을 채울수도 있다. 

,0,1,2
0,-0.007904,0.500000,0.000000
1,-0.419093,0.500000,0.000000
2,1.868058,0.500000,0.603798
3,-1.282961,0.500000,0.621834
4,1.076719,0.452850,-0.995992
5,0.655906,0.804638,-1.080234
6,0.409881,0.493251,-1.179825


In [38]:
df

,0,1,2
0,-0.007904,NaN,NaN
1,-0.419093,NaN,NaN
2,1.868058,NaN,0.603798
3,-1.282961,NaN,0.621834
4,1.076719,0.452850,-0.995992
5,0.655906,0.804638,-1.080234
6,0.409881,0.493251,-1.179825


In [39]:
#fillna는 새로운 객체를 반환하지만 다음처럼 기존 객체를 변경할 수도 있다.
_ = df.fillna(0, inplace=True)  #inplace=True 옵션은 교체되는것을 참으로 처리하여 df값이 변경되도록 한다.

df

,0,1,2
0,-0.007904,0.000000,0.000000
1,-0.419093,0.000000,0.000000
2,1.868058,0.000000,0.603798
3,-1.282961,0.000000,0.621834
4,1.076719,0.452850,-0.995992
5,0.655906,0.804638,-1.080234
6,0.409881,0.493251,-1.179825


In [40]:
df = pd.DataFrame(np.random.randn(6,3))
df.iloc[2:, 1] = NA
df.iloc[4:, 2] = NA
df

,0,1,2
0,-0.651001,0.166969,1.894692
1,0.502763,-0.334765,-0.403801
2,-0.530764,NaN,0.699386
3,-0.095668,NaN,0.071737
4,0.582713,NaN,NaN
5,3.278663,NaN,NaN


In [41]:
df.fillna(method='ffill') #재색인에서 사용 가능한 보간 메소드는 fillna메소드에서도 사용 가능하다.

,0,1,2
0,-0.651001,0.166969,1.894692
1,0.502763,-0.334765,-0.403801
2,-0.530764,-0.334765,0.699386
3,-0.095668,-0.334765,0.071737
4,0.582713,-0.334765,0.071737
5,3.278663,-0.334765,0.071737


In [42]:
#pad, ffill : Nan 값을 앞의 값으로 채운다
#bfill,backfill : Nan 값을 뒤의 값으로 채운다

df.fillna(method='bfill') #ffill 대신에 bfill 사용

,0,1,2
0,-0.651001,0.166969,1.894692
1,0.502763,-0.334765,-0.403801
2,-0.530764,NaN,0.699386
3,-0.095668,NaN,0.071737
4,0.582713,NaN,NaN
5,3.278663,NaN,NaN


In [43]:
df.fillna(method='ffill', limit=2) #limit 옵션을 사용해서 2개까지만 보간

,0,1,2
0,-0.651001,0.166969,1.894692
1,0.502763,-0.334765,-0.403801
2,-0.530764,-0.334765,0.699386
3,-0.095668,-0.334765,0.071737
4,0.582713,NaN,0.071737
5,3.278663,NaN,0.071737


In [44]:
data = pd.Series([1., NA, 3.5, NA, 7])
data

0    1.0
1    NaN
2    3.5
3    NaN
4    7.0
dtype: float64

In [45]:
data.fillna(data.mean()) #Series의 중간값을 전달

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

## 데이터 변형

### 중복 제거하기 
#### 여러가지 이유로 DataFrame에서 중복된 row를 발경할 수있다. 

In [46]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                    'k2': [1,1,2,3,3,4,4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [47]:
data.duplicated() #duplicated 메소드는 각 row가 중복인지 아닌지를 알려주는 Series를 반환한다. 

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [48]:
data.drop_duplicates() #drop_duplicates는 duplicated 배열이 False인 DataFrame을 반환한다.

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


#### 이 두 메소드는 기본적으로 모든 컬럼에 적용되며 중복을 찾아내가 위한 부분합을 따로 지정해줄수도 있다. 

In [49]:
#새로운 컬럼 하나를 추가하고 'k1'컬럼에 기반해 중복을 걸러낸다.
data['v1'] = range(7)
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [50]:
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
1,two,1,1


In [51]:
#duplicated와 drop_duplicates는 기본적으로 처음 발견된 값을 유지한다. 
data.drop_duplicates(['k1', 'k2']) 

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5


In [52]:
#duplicated와 drop_duplicates는 기본적으로 처음 발견된 값을 유지한다. 
#keep='last'옵션을 넘기면 마지막으로 발견된 값을 반환한다.
#data.drop_duplicates(['k1', 'k2']) 
data.drop_duplicates(['k1', 'k2'], keep='last') 

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


### 함수 또는 매핑을 이용해서 데이터 변경하기 
#### DataFrame의 컬럼이나, Series, 배열 내의 값을 기반으로 데이터의 형태를 변화하고자 할때.

In [53]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                             'Pastrami', 'corned beef', 'Bacon',
                            'pastrami', 'honey ham', 'nova lox'],
                    'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [54]:
#해당 육류가 어떤 동물의 고기인지 알려주는 컬럼을 추가한다고 가정하자.
meat_to_animal = {
    'bacon': 'pig',
    'pulled pork': 'pig',
    'pastrami': 'cow',
    'corned beef': 'cow',
    'honey ham': 'pig',
    'nova lox': 'salmon'
}

In [55]:
lowercased = data['food'].str.lower() #데이터에 대소문자 섞여있으므로, 소문자로 변경
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [56]:
#Series의 map 메소드는 사전류의 객체나 어떤 함수를 받을 수 있다.
data['animal'] = lowercased.map(meat_to_animal) #map 메소드 이용
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [57]:
data['food'].map(lambda x: meat_to_animal[x.lower()]) #함수를 넘겨서 같은 일을 수행 가능

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

### 값 치환하기 
#### fillna같이 누락된 값을 채우는 일은 일반적인 치환 작업이다. 
#### 위애서 map 메소드를 한 객체안에서 값의 부분집합을 변경하는데 사용하였다.
#### replace 메소드는 같은 작업에 대해 보다 간단하고 유연한 방법을 제공한다. 

In [59]:
data = pd.Series([1., -999., 2., -999., 3.])
data

0      1.0
1   -999.0
2      2.0
3   -999.0
4      3.0
dtype: float64

In [60]:
data.replace(-999, np.nan) #replace를 이용한 치환

0    1.0
1    NaN
2    2.0
3    NaN
4    3.0
dtype: float64

In [61]:
data.replace([-999, 1.0], np.nan) #여러개의 값을 치환하려면 값의 리스트를 넘기면 된다.

0    NaN
1    NaN
2    2.0
3    NaN
4    3.0
dtype: float64

In [45]:
data.replace([-999, 1.0], [np.nan, 0]) #치환하려는 값마다 다른 값으로 치환하려면 새로 지정할 리스트를 넘긴다.

0    0.0
1    NaN
2    2.0
3    NaN
4    3.0
dtype: float64

In [62]:
data.replace({-999: np.nan, -1000: 0}) #리스트 대신 사전을 사용하는 것도 가능하다

0    1.0
1    NaN
2    2.0
3    NaN
4    3.0
dtype: float64

### 축 색인 이름 바꾸기 
#### Series의 값들처럼 축 이름 역시 유사한 방식으로 바꿀 수 있다. 

In [70]:
data = pd.DataFrame(np.arange(12).reshape((3,4)),
                   index = ['Ohio', 'Colorado', 'New York'],
                   columns = ['one', 'two', 'three', 'four'])

In [71]:
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [72]:
transform = lambda x: x[:4].upper()

data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

In [73]:
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [74]:
data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [75]:
data.rename(index=str.title, columns=str.upper) #원래 객체를 변경하지 않고 새로운 객체를 생성하려면 rename 메소드를 사용한다.

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


In [54]:
data.rename(index={'OHIO':"INDIANA"},
           columns = {'three':'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [55]:
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [56]:
#rename 메소드를 사용하면 DataFrame을 직접 복사해서 index와 column 속성을 갱신할 필요없이 바로 변경 가능
#원본 데이터를 바로 변경하려면, inplace=True 옵션을 사용하면 된다.
data.rename(index={'OHIO': 'INDIANA'}, inplace=True) 
data

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [68]:
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


### 개별화와 양자화 
#### 연속성 데이터는 종종 개별로 분할 하거나 아니면 분석을 위해 그룹별로 나누기도 한다. 

In [78]:
#수업에 참여하는 학생 그룹 데이터가 있고, 나이에 따라 분류한다고 가정하자.
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]
bins = [18, 25, 35, 60, 100]

cats = pd.cut(ages, bins) #cut 함수를 이용해서 18-25, 26-35, 36-60, 61-100 그룹으로 나눈다.
cats
#pandas에서 반환하는 객체는 Categorical이라는 특수한 객체이다. 이 객체는 그룹 이름이 담긴 배열이라고 생각하면 된다. 

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64, right]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [79]:
cats.codes  #4개의 categorie중 어디에 속하는지 표시

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [80]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]], dtype='interval[int64, right]')

In [81]:
pd.value_counts(cats)

(18, 25]     5
(25, 35]     3
(35, 60]     3
(60, 100]    1
dtype: int64

#### 간격을 나타내는 표기법은 중괄로 시작해서 대괄호로 끝난다. 
#### 중괄호 쪽의 값은 포함하지 않고 대괄호쪽의 값은 포함한다. 
#### right=Flase 를 사용해서 중괄호 대신 대괄호쪽이 포함되지 않게 바꿀 수 있다.

In [82]:
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64, left]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

In [83]:
#labels 옵션을 사용해서 그룹의 이름을 직접 넘겨줄 수도 있다. 
group_names = ['Youth', 'YoungAdult', 'MiddleAges', 'Senior']
pd.cut(ages, bins, labels=group_names)

['Youth', 'Youth', 'Youth', 'YoungAdult', 'Youth', ..., 'YoungAdult', 'Senior', 'MiddleAges', 'MiddleAges', 'YoungAdult']
Length: 12
Categories (4, object): ['Youth' < 'YoungAdult' < 'MiddleAges' < 'Senior']

#### 만약 cut함수에 명시적으로 그룹의 경계값을 넘기지 않고 그룹의 갯수를 넘겨주면, 데이터의 최소값과 최대값을 기준으로 자동으로 균등한 길이의 그룹을 자동 생성한다. 

In [84]:
#어떤 균등분포에서 4개의 그룹으로 나누는 경우
data = np.random.rand(20)
data

array([0.34532307, 0.84482903, 0.31816258, 0.21209649, 0.3820675 ,
       0.27358927, 0.68082503, 0.31921454, 0.2238648 , 0.23583255,
       0.17555642, 0.46400657, 0.68347753, 0.38336386, 0.27306336,
       0.4574661 , 0.45569634, 0.37755203, 0.28924082, 0.32807021])

In [85]:
pd.cut(data, 4, precision=2) #precision=2 옵션은 소수점 아래 2자리까지 제한한다.

[(0.34, 0.51], (0.68, 0.84], (0.17, 0.34], (0.17, 0.34], (0.34, 0.51], ..., (0.34, 0.51], (0.34, 0.51], (0.34, 0.51], (0.17, 0.34], (0.17, 0.34]]
Length: 20
Categories (4, interval[float64, right]): [(0.17, 0.34] < (0.34, 0.51] < (0.51, 0.68] < (0.68, 0.84]]

In [86]:
data = pd.cut(data, 4, precision=2) 
pd.value_counts(data)

(0.17, 0.34]    10
(0.34, 0.51]     7
(0.68, 0.84]     3
(0.51, 0.68]     0
dtype: int64

#### cut 함수와 유사한 qcut함수가 있다. 
#### qcut함수는 표본 변위치를 기준으로 나눈다. cut함수를 사용하면 데이터의 분산에 따라 각각의 그룹마다 데이터 수가 다르게 나누어지는데,
#### qcut함수는 표준 변위치를 사용하기 때문에 적당히 같은 크기의 그룹으로 나눌 수 있다.

In [87]:
data = np.random.randn(1000)
cats = pd.qcut(data, 4)
cats

[(-0.00755, 0.619], (-0.666, -0.00755], (-3.022, -0.666], (-0.666, -0.00755], (0.619, 3.434], ..., (-3.022, -0.666], (-0.00755, 0.619], (-0.00755, 0.619], (-0.00755, 0.619], (0.619, 3.434]]
Length: 1000
Categories (4, interval[float64, right]): [(-3.022, -0.666] < (-0.666, -0.00755] < (-0.00755, 0.619] < (0.619, 3.434]]

In [88]:
pd.value_counts(cats)

(-3.022, -0.666]      250
(-0.666, -0.00755]    250
(-0.00755, 0.619]     250
(0.619, 3.434]        250
dtype: int64

In [89]:
cats = pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.]) #cut함수와 유사하게 변위치를 직접지정 가능(변위치는 0에서 1까지)

In [90]:
pd.value_counts(cats )

(-1.228, -0.00755]    400
(-0.00755, 1.237]     400
(-3.022, -1.228]      100
(1.237, 3.434]        100
dtype: int64

### 특이값(outlier) 검출 및 제외 

In [96]:
data = pd.DataFrame(np.random.randn(1000, 4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.057382,0.014833,-0.032837,-0.070610
std,0.993176,0.969879,1.000408,1.006337
min,-2.903007,-3.208152,-3.159258,-3.066049
25%,-0.611538,-0.658639,-0.742242,-0.764255
50%,-0.005104,0.035318,-0.037915,-0.034909
75%,0.750757,0.596034,0.671206,0.607393
max,3.871073,3.034368,3.440961,2.972842


In [97]:
data

,0,1,2,3
0,-0.435332,1.844605,-0.058965,1.451069
1,-0.853249,0.364704,-1.005453,-0.086275
2,0.183970,-0.510314,-3.026479,1.166023
3,-1.193857,0.382629,0.806334,0.919959
4,-1.246482,0.189645,0.756360,-0.545974
...,...,...,...,...
995,-0.824161,-0.374222,-1.018969,-0.283182
996,-0.939143,0.697190,2.566374,-1.312721
997,-0.967981,-0.909285,1.066371,-0.876593
998,-0.386709,-1.197742,-2.352593,-0.390545


In [99]:
col = data[2]
col[np.abs(col) > 3] #DataFrame의 특정 한 컬럼(data[2])에서 3을 초과하는 값을 찾는 경우

2     -3.026479
327    3.440961
695   -3.159258
Name: 2, dtype: float64

In [100]:
data[(np.abs(data) > 3).any(1)] #3을 초과하는 모든 row를 선택하려면 any 메소드를 사용

,0,1,2,3
2,0.183970,-0.510314,-3.026479,1.166023
51,3.262043,-3.208152,-0.280390,-0.195515
96,0.640876,-0.439869,1.032299,-3.066049
201,2.482059,3.027633,-0.478040,0.360770
327,0.509991,0.218871,3.440961,-1.061086
533,-0.439013,-3.123027,0.708493,-0.292496
540,3.347523,-0.416249,0.912796,-2.666492
581,3.871073,-0.365977,-0.413748,-0.374311
695,2.161193,-0.858941,-3.159258,0.022551
910,-0.416318,3.034368,0.040269,-0.826572


In [101]:
# Capping outside -3 to 3
data[np.abs(data) > 3] = np.sign(data) * 3  #-3이나 3을 초과하는 값을 -3 또는 3으로 지정, np.sign(data)는 부호에 따라 1이나 -1을 반환
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.055902,0.015102,-0.033093,-0.070544
std,0.988294,0.968625,0.998406,1.006142
min,-2.903007,-3.000000,-3.000000,-3.000000
25%,-0.611538,-0.658639,-0.742242,-0.764255
50%,-0.005104,0.035318,-0.037915,-0.034909
75%,0.750757,0.596034,0.671206,0.607393
max,3.000000,3.000000,3.000000,2.972842


In [102]:
data.head() #head는 기본으로 5개 출력, head(10)이면 10개 출력

,0,1,2,3
0,-0.435332,1.844605,-0.058965,1.451069
1,-0.853249,0.364704,-1.005453,-0.086275
2,0.183970,-0.510314,-3.000000,1.166023
3,-1.193857,0.382629,0.806334,0.919959
4,-1.246482,0.189645,0.756360,-0.545974


In [103]:
np.sign(data).head()

,0,1,2,3
0,-1.0,1.0,-1.0,1.0
1,-1.0,1.0,-1.0,-1.0
2,1.0,-1.0,-1.0,1.0
3,-1.0,1.0,1.0,1.0
4,-1.0,1.0,1.0,-1.0


### 치환(permutation)과 임의 샘플링 
#### nimpy.random.permutation함수를 사용하면 row를 쉽게 임의 순서로 재배치 할 수있다.

In [104]:
df = pd.DataFrame(np.arange(5*4).reshape((5,4)))
sampler = np.random.permutation(5)
sampler

array([4, 3, 0, 1, 2])

In [105]:
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [106]:
df.take(sampler) #take 함수를 사용

,0,1,2,3
4,16,17,18,19
3,12,13,14,15
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11


In [107]:
df.sample(n=3) #치환 없이 일부만 임의로 선택하려면 sample 메소드 사용

,0,1,2,3
0,0,1,2,3
2,8,9,10,11
3,12,13,14,15


In [108]:
choices = pd.Series([5, 7, -1, 6, 4])
draws = choices.sample(n=10, replace=True) #반복선택을 허용하며, 표본을 치환을 통해 생성해내려면 replace=True 옵션 사용
draws

2   -1
3    6
0    5
0    5
3    6
3    6
4    4
2   -1
1    7
1    7
dtype: int64

### 표시자/더미(dummy) 변수 생성하기 
#### 통계나 머신러닝 에플리케이션을 위해 분류값을 '더미(dummy)'나 '표시자(indicator)' 행렬로 전환한다. 
#### 한 컬럼에 k가지의 값이 있다면, k개의 컬럼이 있는 행렬을 만들고 값으로는 1과 0을 채워 넣는다. 

In [109]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                  'data1': range(6)})
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [110]:
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [111]:
dummies = pd.get_dummies(df['key'], prefix='key') #컬럼에 접두어(prefix)를 추가한 후 다른 데이터와 병합, prefix인자 사용
dummies


,key_a,key_b,key_c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [112]:
df_with_dummy = df[['data1']].join(dummies) 
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


#### DataFrame의 한 row가 여러 카테고리에 속하다면 조금 복잡해진다. 
#### 'MovieLens의 영화 평점 데이터' 에서 살펴본다. 

In [120]:
import chardet
filename = 'datasets/movielens/movies.dat'
with open(filename, 'rb') as f:
    result = chardet.detect(f.readline())
    print(result['encoding'])

ascii


In [121]:
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('datasets/movielens/movies.dat', sep='::',
                      header=None, names=mnames)
movies[:10]

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [125]:
#각 장르마다 표시자 값을 추가하기위해 먼저 유일한 장르 목록을 추출한다.
all_genres = []
for x in movies.genres:
    all_genres.extend(x.split('|'))
    
genres = pd.unique(all_genres)
genres

array(['Animation', "Children's", 'Comedy', 'Adventure', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Sci-Fi', 'Documentary', 'War', 'Musical', 'Mystery', 'Film-Noir',
       'Western'], dtype=object)

In [126]:
zero_matrix = np.zeros((len(movies), len(genres)))
dummies = pd.DataFrame(zero_matrix, columns=genres) #표시자 DataFrame을 생성하기 위해 0으로 초기화된 DataFrame 생성

In [124]:
dummies

,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3878,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3879,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3880,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3881,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [127]:
gen = movies.genres[0]  #첫번째 영화의 장르를 '|'를 기준으로 구분한다.
gen.split('|')

['Animation', "Children's", 'Comedy']

In [128]:
dummies.columns.get_indexer(gen.split('|'))

array([0, 1, 2], dtype=int64)

In [129]:
for i, gen in enumerate(movies.genres):
    indices = dummies.columns.get_indexer(gen.split('|'))
    dummies.iloc[i, indices] = 1

In [130]:
movies_windic = movies.join(dummies.add_prefix('Genre_'))
movies_windic.iloc[1]

movie_id                                        2
title                              Jumanji (1995)
genres               Adventure|Children's|Fantasy
Genre_Animation                               0.0
Genre_Children's                              1.0
Genre_Comedy                                  0.0
Genre_Adventure                               1.0
Genre_Fantasy                                 1.0
Genre_Romance                                 0.0
Genre_Drama                                   0.0
Genre_Action                                  0.0
Genre_Crime                                   0.0
Genre_Thriller                                0.0
Genre_Horror                                  0.0
Genre_Sci-Fi                                  0.0
Genre_Documentary                             0.0
Genre_War                                     0.0
Genre_Musical                                 0.0
Genre_Mystery                                 0.0
Genre_Film-Noir                               0.0


#### get_dummies와 cut 같은 이산함수를 잘 조합하면 통계 애플리케이션에서 유용하게 사용할 수 있다. 

In [131]:
np.random.seed(12345)
values = np.random.rand(10)
values

array([0.92961609, 0.31637555, 0.18391881, 0.20456028, 0.56772503,
       0.5955447 , 0.96451452, 0.6531771 , 0.74890664, 0.65356987])

In [132]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
pd.cut(values, bins)

[(0.8, 1.0], (0.2, 0.4], (0.0, 0.2], (0.2, 0.4], (0.4, 0.6], (0.4, 0.6], (0.8, 1.0], (0.6, 0.8], (0.6, 0.8], (0.6, 0.8]]
Categories (5, interval[float64, right]): [(0.0, 0.2] < (0.2, 0.4] < (0.4, 0.6] < (0.6, 0.8] < (0.8, 1.0]]

In [133]:
pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,0,0,0,0,1
1,0,1,0,0,0
2,1,0,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0
5,0,0,1,0,0
6,0,0,0,0,1
7,0,0,0,1,0
8,0,0,0,1,0
9,0,0,0,1,0


## 문자열 다루기 

### 문자열 객체 메소드 
#### 문자열을 다루어야 하는 대부분의 에플리케이션은 내장 문자열 메소드만으로도 충분하다. 

In [134]:
val = 'a,b,   guido'

val.split(',')  #,로 구분된 문자열은 split 메소드를 이용해 분리

['a', 'b', '   guido']

In [135]:
pieces = [x.strip() for x in val.split(',')] #split은 종종 공백문자(줄바꿈 문자 포함)를 제거하는 strip과 조합해서 사용한다.
pieces  #출력결과 공백이 제거된것을 확인

['a', 'b', 'guido']

In [136]:
first, second, third = pieces
first + '::' + second + '::' + third

'a::b::guido'

In [137]:
'::'.join(pieces) #위의 방법보다 편리하고 파이썬스러운 방법, 리스트나 튜플을 ::문자열의 join메소드로 전달하는것이다.

'a::b::guido'

#### 일치하는 부분문자열의 위치를 찾는 방법도 있다. index나 find를 사용하는것도 가능하지만 
#### 파이썬의 in 예약어를 사용하면 일치하는 문자열을 쉽게 찾을 수 있다. 

In [138]:
'guido' in val

True

In [139]:
val.index(',') #index의 경우 찾으면 1을 반환

1

In [140]:
val.find(':') #find의 경우 찾지 못하면 -1을 반환

-1

In [141]:
val.index(':') #index의 경우 찾지 못하면 예외를 발생한다. 

ValueError: substring not found

In [142]:
val.count(',')

2

In [143]:
val.replace(',', '::')  #replace는 찾는 퍄턴을 다른 문자열로 치환한다.

'a::b::   guido'

In [144]:
val.replace(',', '')

'ab   guido'

### 정규 표현식
#### 파이썬에는 re 모듈이 내장되어 있어 문자열에 대한 정규 표현식을 처리한다. 

In [145]:
import re

text = "foo bar\t baz   \tqux" #여러가지 공백문자(탭, 스페이스, 개행문자)가 포함된 문자열
text

'foo bar\t baz   \tqux'

In [146]:
re.split('\s+', text)   #'\s+'는 하나 이상의 공백문자를 의미한다.
#정규 표현식이 먼저 컴파일되고, 그 다음 split 메서드가 실행된다.

['foo', 'bar', 'baz', 'qux']

In [147]:
regex = re.compile('\s+') #먼저 컴파일
regex.split(text)         #정규표현식 객체 재사용

['foo', 'bar', 'baz', 'qux']

In [148]:
regex.findall(text) #정규 표현식에 매칭되는 모든 패턴의 목록을 얻고 싶다면 findall 메소드 사용

[' ', '\t ', '   \t']

#### match와 search는 findall 메서드와 관련이 있다.
#### findall : 문자열에서 일치하는 모든 부분 문자열을 찾아준다. 
#### search : 패턴과 일치하는 첫 번째 존재를 반환한다. 
#### match : 문자열의 시작부분에서 일치하는 것만 찾아준다 
#### sub : 찾은 패턴을 주어진 문자열로 치환하여 새로운 문자열을 반환

In [149]:
#이메일 주소를 검사하는 정규 표현식
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
jwseok@changwon.ac.kr
"""

pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'
regex = re.compile(pattern, flags=re.IGNORECASE) #re.IGNORECASE는 정규 표현식이 대소문자를 가리지 않도록 한다.

In [150]:
regex.findall(text)

['dave@google.com',
 'steve@gmail.com',
 'rob@gmail.com',
 'ryan@yahoo.com',
 'jwseok@changwon.ac.kr']

In [151]:
m = regex.search(text)  #패턴과 일치하는 찻반째 존재
m

<re.Match object; span=(5, 20), match='dave@google.com'>

In [152]:
text[m.start():m.end()]

'dave@google.com'

In [153]:
print(regex.match(text))

None


In [160]:
r = re.compile("[ab]")
print(r.search("pizza"))
print(r.match("pizza"))
print(r.match("abcd"))

<re.Match object; span=(4, 5), match='a'>
None
<re.Match object; span=(0, 1), match='a'>


In [161]:
print(regex.sub('REDACTED', text))

Dave REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED
REDACTED



In [162]:
#이메일 주소를 각 콤포논트별(사용자이름, 도메인이름, 도메인접미사)로 나누어야 한다면, 각 패턴을 괄호로 묶어준다.
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})' 
regex = re.compile(pattern, flags=re.IGNORECASE)

In [163]:
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com'),
 ('jwseok', 'changwon.ac', 'kr')]

In [24]:
m = regex.match('wes@bright.net')
m.groups()  #group메서드로 각 패넌 컴포넌트의 튜플을 얻을 수 있다.

('wes', 'bright', 'net')

In [25]:
#sub 은 \1, \2와 같은 특수한 기호를 사용하여 각 패턴그룹에 접근할 수 있다.
#\1은 첫 번째, \2는 두 번째 그룹을 의미한다.
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text))

Dave Username: dave, Domain: google, Suffix: com
Steve Username: steve, Domain: gmail, Suffix: com
Rob Username: rob, Domain: gmail, Suffix: com
Ryan Username: ryan, Domain: yahoo, Suffix: com



### 벡터화된 문자열 함수

In [164]:
import pandas as pd
import numpy as np
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
        'Rob': 'rob@gmail.com', 'Wes': np.nan}
data = pd.Series(data)
data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

In [31]:
data.isnull()

Dave     False
Steve    False
Rob      False
Wes       True
dtype: bool

#### 문자열고 정규표현식 메서는 data.map을 사용
#### 하지만, NA 값을 만나면 실패하게 된다. 
#### 이 경우 Series에서는 str속성을 이용해서 처리한다. 
#### 정규표현식을 'IGNORECASE' 같은 re 옵션과 함께 사용하는 것도 가능하다. 

In [32]:
data.str.contains('gmail')

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

In [33]:
pattern

'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\\.([A-Z]{2,4})'

In [34]:
data.str.findall(pattern, flags=re.IGNORECASE)

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

#### To retrieve elements from Vector, we can either use 'str.get' or 'index'.
#### To access elements in embedded list, we pass an index to either of these functions.
#### We can also slice strings with this syntax.

In [37]:
matches = data.str.match(pattern, flags=re.IGNORECASE)
matches

Dave     True
Steve    True
Rob      True
Wes       NaN
dtype: object

In [41]:
data.str[:5]  #문자열을 잘라낼 수 있다.

Dave     dave@
Steve    steve
Rob      rob@g
Wes        NaN
dtype: object